# Actualizacion de Tablas SQL
## Utilizando las nuevas tablas desde el modulo de sistematizacion

Antes que todo, es necesario que la carpeta `db/CSV/` se encuentra con los archivos nuevos -i.e., copiar y reemplazar en la carpeta las tablas nuevas.

En particular, las nuevas tablas contienen procesada la columna `macro`. Esta columna asigna una etiqueta o categoria, permitiendo agrupar distintas respuestas. 

In [1]:
import pandas as pd
import numpy as np
import os 


from use_cases.pairs import create_pair_token
from use_cases.utils.textools import clean_alt_list, to_unicode, str_to_int, eliminate_nrs

from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Emociones y Sentimientos

In [2]:
emotions_table = pd.read_csv('./db/CSV/emotions.csv', low_memory=False)

In [3]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
75832,75832,NaN,3106330,impotencia,['impotencia'],Tristeza,de no poder hacer nada por cambiar el pais a l...,"['poder', 'hacer', 'cambiar', 'pais', 'edad']",False
54535,54535,enc_u_4598366420779259830,NaN,rabia,['rabia'],Enojo,por lo que generan los destrozos y no se puede...,"['generan', 'destrozos', 'puede', 'hacer']",True


las listas se guardan como `strings` en el csv, es necesario volver a dejarlos como `listas`, asi tambien como pasar los id de los dialogos a `unicode`

In [4]:
emotions_table = eliminate_nrs(emotions_table)
emotions_table['name_tokens'] = emotions_table['name_tokens'].apply(lambda x: clean_alt_list(x))
emotions_table['exp_tokens'] = emotions_table['exp_tokens'].apply(lambda x: clean_alt_list(x))

In [5]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
64303,64303,,3192592,incomodidad,[incomodidad],Enojo,por que veo que no se soluciona nada,"[veo, soluciona]",False
50231,50231,enc_u_4574883470112197190,,incertidumbre,[incertidumbre],Miedo,no se sabe lo que seguira,"[sabe, seguira]",True


Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [6]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos `.sql`

In [7]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv', index=False)
emo_to_sql(emotions_table, './db/SQL_INSERT/emotions.sql')

### Contribuciones

En el caso de contribuciones solo necesitamos guardar el nuevo `csv` como `sql`

In [8]:
contributions_table = pd.read_csv('./db/CSV/contributions.csv', low_memory=False)

In [9]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
64658,79821,NaN,4724252,trabajar para el estado,['trabajar'],Trabajo,False


In [10]:
contributions_table = eliminate_nrs(contributions_table)
contributions_table['tokens'] = contributions_table['tokens'].apply(lambda x: clean_alt_list(x))

In [11]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
63725,78329,,5540287,que el gobierno escuche al pais y no que esper...,"[gobierno, escuche, pais, esperen, chile, term...",Voluntariado,False


In [12]:
con_to_sql(contributions_table, './db/SQL_INSERT/contributions.sql')

### Necesidades Personales

In [14]:
personal_needs_table = pd.read_csv('./db/CSV/personal_needs.csv', low_memory=False)

In [15]:
personal_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
14169,14169,enc_u_4542244700112003015,NaN,inseguridad,['inseguridad'],los barrios estan tomados por la delincuencia ...,"['barrios', 'estan', 'tomados', 'delincuencia'...",vivienda,1,False


In [16]:
personal_needs_table = eliminate_nrs(personal_needs_table)

personal_needs_table['name_tokens'] = personal_needs_table['name_tokens'].apply(lambda x: clean_alt_list(x))
personal_needs_table['exp_tokens'] = personal_needs_table['exp_tokens'].apply(lambda x: clean_alt_list(x))


personal_needs_table['is_online'] = personal_needs_table['is_online'].apply(lambda x: bool(x))
personal_needs_table['priority'] = personal_needs_table['priority'].apply(lambda x: int(x))

In [17]:
personal_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
86745,86745,,4214867,salud,[salud],hay medicamentos que no entregan en el consult...,"[medicamentos, entregan, consultorio, salud, m...",salud,3,False


Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [18]:
personal_needs_pairs = create_pair_token(personal_needs_table, 'exp_tokens', 'personal_need_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos `.sql`

In [19]:
personal_needs_pairs.to_csv('./db/CSV/personal_needs_pairs.csv', index=False)
per_need_to_sql(personal_needs_table, './db/SQL_INSERT/personal_needs.sql')

### Necesidades Pais

In [20]:
country_needs_table = pd.read_csv('./db/CSV/country_needs.csv', low_memory=False)

In [21]:
country_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
16907,16908,enc_u_4520740619151564403,NaN,falta de dinero,"['falta', 'dinero']",NR,NaN,NR,aumento de pensiones\n-aumento de sueldos\n-au...,"['aumento', 'pensiones', 'aumento', 'sueldos',...",estado,NaN,False


In [22]:
country_needs_table = eliminate_nrs(country_needs_table)
country_needs_table['name_tokens'] = country_needs_table['name_tokens'].apply(lambda x: clean_alt_list(x))
country_needs_table['exp_tokens'] = country_needs_table['exp_tokens'].apply(lambda x: clean_alt_list(x))
country_needs_table['role_tokens'] = country_needs_table['role_tokens'].apply(lambda x: clean_alt_list(x))

country_needs_table['is_online'] = country_needs_table['is_online'].apply(lambda x: bool(x))
country_needs_table['priority'] = country_needs_table['priority'].apply(lambda x: str_to_int(x))

In [23]:
country_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
1876,1877,enc_u_4549115135187566849,,salud,[salud],salud,- acceso equitativo\n- alimentacion saludable\...,"[acceso, equitativo, alimentacion, saludable, ...",consultorios mayor integracion a poblacion mig...,"[consultorios, mayor, integracion, poblacion, ...",consultorios,0,False


Debemos recalcular los pares de palabras en caso de que algo haya cambiado, en este caso tenemos dos tablas: exp y role

In [24]:
country_needs_exp_pairs_table = create_pair_token(country_needs_table, 'exp_tokens', 'country_need_id')
country_need_role_pair_table = create_pair_token(country_needs_table, 'role_tokens', 'country_need_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos .sql

In [25]:
country_needs_exp_pairs_table.to_csv('./db/CSV/country_needs_exp_pairs.csv', index=False)
country_need_role_pair_table.to_csv('./db/CSV/country_needs_role_pairs.csv', index=False)
count_need_to_sql(country_needs_table, './db/SQL_INSERT/country_needs.sql')